In [6]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import f1_score
import numpy as np
from tqdm.notebook import tqdm, trange
torch.manual_seed(1)
all_data=pd.read_csv('WikiLarge_Train.csv')
#from sklearn.model_selection import StratifiedShuffleSplit
#sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
#for train_index, test_index in sss.split(all_data, all_data['label']):
#    train_df=all_data.iloc[train_index]
#    dev_df=all_data.iloc[test_index]
#train_df['label'].value_counts()/len(train_df)
#vectorizer = CountVectorizer()


In [2]:
#train_df.head()

,original_text,label
401297,"Even when their burrow is underwater , mudskip...",0
347368,The first documented mention of Hungen is in t...,0
325250,"1921 - Karel Appel , Dutch painter",0
3693,Both GQ and In Style had Alba on their June 20...,1
246480,Design The most well-known type of electrode i...,0


In [3]:
X_train = vectorizer.fit_transform(train_df.original_text).todense()
X_dev  = vectorizer.transform(dev_df.original_text).todense()

y_train = list([int(y) for y in train_df['label']])
y_dev  = list([int(y) for y in dev_df['label']])

In [54]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score

def result_evalation(predict_result, true_result):
    accuracy=accuracy_score(true_result, predict_result)
    precision=precision_score(true_result, predict_result)
    recall=recall_score(true_result, predict_result)
    f1=f1_score(true_result, predict_result)
    auc=roc_auc_score(true_result, predict_result)
    return accuracy,precision,recall, f1, auc
    


In [ ]:
def plot_roc(predict_results, true_label):
    from sklearn import metrics
    import pylab as plt
    fpr, tpr, threshold = metrics.roc_curve(predict_results, true_label)
    roc_auc = metrics.auc(fpr, tpr)
    plt.figure(figsize=(6,6))
    plt.title('ROC curve for validation set')
    plt.plot(fpr, tpr, 'b', label = 'Val AUC = %0.3f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [8]:
def to_tensors(instance, label):
    '''Given some vectorized text and the integer label for its class, returns a tuple of these in Tensor format'''
    instance = torch.Tensor(instance)
    label = torch.LongTensor([int(label)])
    return (instance, label)



In [5]:
training_data = [to_tensors(row, y_train[i]) for i, row in enumerate(X_train)]
dev_data = [to_tensors(row, y_dev[i]) for i, row in enumerate(X_dev)]

In [9]:
class LogisticRegression(nn.Module):
    def __init__(self, vocab_size, num_classes=2):
        super(LogisticRegression, self).__init__()
        self.betas = nn.Linear(vocab_size, num_classes)  
    def forward(self, feature_vector):
        return self.betas(feature_vector)

In [61]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True)
kf.get_n_splits(all_data)
all_accuracy=[]
all_recall=[]
all_precision=[]
all_f1=[]
all_auc=[]
for train_index, test_index in kf.split(all_data):
    print("TRAIN:", len(train_index), "Valid:", len(test_index))
    train_df = all_data.iloc[train_index]
    dev_df=all_data.iloc[test_index]
    X_train = vectorizer.fit_transform(train_df.original_text).todense()
    X_dev  = vectorizer.transform(dev_df.original_text).todense()
    y_train = list([int(y) for y in train_df['label']])
    y_dev  = list([int(y) for y in dev_df['label']])
    training_data = [to_tensors(row, y_train[i]) for i, row in enumerate(X_train)]
    dev_data = [to_tensors(row, y_dev[i]) for i, row in enumerate(X_dev)]
    loss_function = nn.CrossEntropyLoss()
    vocab_size = len(vectorizer.vocabulary_)
    model = LogisticRegression(vocab_size)
    learning_rate = 5e-5
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 
    for epoch in trange(1, desc='Epoch'):
        for (instance, label) in tqdm(training_data, leave=False, desc='Training Loop for Epoch %d' % epoch):
            model.zero_grad()        
            output = model(instance)
            loss = loss_function(output, label)
            loss.backward()
            optimizer.step()
    dev_preds = []
    with torch.no_grad():
        for (instance, label) in dev_data:
            outputs = model(instance)
            dev_preds.append(np.argmax(outputs))
    accuracy_each, precsion_each, recall_each, f1_each, auc_reach=result_evalation(dev_preds,y_dev)
    all_accuracy.append(accuracy_each)
    all_precision.append(precsion_each)
    all_recall.append(recall_each)
    all_f1.append(f1_each)
    all_auc.append(auc_reach)   

TRAIN: 333414 Valid: 83354


/sw/arcts/centos7/python3.8-anaconda/2021.05/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TRAIN: 333414 Valid: 83354


/sw/arcts/centos7/python3.8-anaconda/2021.05/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TRAIN: 333414 Valid: 83354


/sw/arcts/centos7/python3.8-anaconda/2021.05/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TRAIN: 333415 Valid: 83353


/sw/arcts/centos7/python3.8-anaconda/2021.05/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TRAIN: 333415 Valid: 83353


/sw/arcts/centos7/python3.8-anaconda/2021.05/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [49]:
np.mean(all_accuracy)

0.645

In [51]:
np.mean(all_recall)

0.589

In [53]:
np.mean(all_auc)

0.644

In [20]:
from sklearn.model_selection import StratifiedShuffleSplit
sss= StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(all_data, all_data['label']):
    train_df=all_data.iloc[train_index]
    dev_df=all_data.iloc[test_index]
train_df['label'].value_counts()/len(train_df)
vectorizer = CountVectorizer()
train_df = all_data.iloc[train_index]
dev_df=all_data.iloc[test_index]
X_train = vectorizer.fit_transform(train_df.original_text).todense()
X_dev  = vectorizer.transform(dev_df.original_text).todense()
y_train = list([int(y) for y in train_df['label']])
y_dev  = list([int(y) for y in dev_df['label']])


In [42]:
training_data = [to_tensors(row, y_train[i]) for i, row in enumerate(X_train)]
dev_data = [to_tensors(row, y_dev[i]) for i, row in enumerate(X_dev)]

In [43]:
len(y_dev)

83354

In [23]:

loss_function = nn.CrossEntropyLoss()


vocab_size = len(vectorizer.vocabulary_)


model = LogisticRegression(vocab_size)


learning_rate = 5e-5


optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 


for epoch in trange(3, desc='Epoch'):

    for (instance, label) in tqdm(training_data, leave=False, desc='Training Loop for Epoch %d' % epoch):

 
        model.zero_grad()        
 
        output = model(instance)
        
 
        loss = loss_function(output, label)
        
      
        loss.backward()
        optimizer.step()

In [45]:
dev_preds=[]
with torch.no_grad():
    for (instance, label) in dev_data:
        outputs = model(instance)
        dev_preds.append(np.argmax(outputs))

In [46]:
len(dev_preds)

83354

In [55]:
accuracy, precision, recall, f1, auc=result_evalation(dev_preds,y_dev)

In [56]:
accuracy

0.6489310650958562

In [57]:
precision

0.6668010319251855

In [58]:
recall

0.5953643496412889

In [59]:
f1

0.6290610858294565

In [60]:
auc

0.6489310650958563

In [29]:
test_data=pd.read_csv('WikiLarge_Test.csv')

In [20]:
X_test  = vectorizer.transform(test_data.original_text).todense()

In [21]:
test_data = [to_tensors(row, 0) for i, row in enumerate(X_test)]

In [22]:

test_preds = []


with torch.no_grad():
    
    
    for (instance, label) in test_data:
        
   
        outputs = model(instance)
        

        test_preds.append(np.argmax(outputs))

In [34]:
test_preds

[tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(0),
 tensor(1),
 tensor(1),
 ten

In [35]:
test_preds_new=[]
for i in test_preds:
    test_preds_new.append(int(i))


In [36]:
sum(test_preds_new)

53486

In [37]:
result=pd.DataFrame({'id':list(test_data['id']), 'label':test_preds_new})
result.to_csv('results_LR.csv',index=None)

In [38]:
sum(result['label'])

53486